In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote
import re

def parse_children_page(url):
    response = requests.get(url)
    
    # Перевірка на успішність відповіді
    if response.status_code == 200:
        # Парсимо HTML за допомогою BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Знаходимо елементи, які містять інформацію (з актуальним класом)
        code = soup.find_all(class_="container-fluid passport")  # Актуалізуйте клас, якщо він інший

        # Перебір знайдених елементів у code
        for element in code:
            # Знаходимо ім'я
            name = element.find(class_="passport__title-name")
            if name:
                print(name.get_text(strip=True))
            else:
                print("Ім'я не знайдено в цьому елементі.")                
            # passport__title-number
            #id
            id = element.find(class_="passport__title-number").get_text(strip=True)
            number = re.search(r'\d+', id).group()
            
            print("id дитини:",number)
            
            # Виводимо фото
            photo = element.find(class_="js-lightBoxGallery").find('img').get('src')
            if photo:
                print("photo src=",photo)
            else:
                print("Фото не знайдено в цьому елементі.")

            #region
            region = element.find("ul").find_all("li")[1]
            if region:
                print("Region:",region.find_all("span")[1].get_text(strip=True))
            else:
                print("region not fount")
            
            #siblings
            siblings = element.find("ul").find_all("li")[2]
            if siblings:
                print("Siblings:",siblings.find_all("span")[1].get_text(strip=True))
            else:
                print("siblings not fount")

            #отримати кількість
            if (siblings.find_all("span")[1].get_text(strip=True) == "є"):
                siblingsCount = element.find(class_='passport__content-inner--brother').find_all("li")
                print("siblings count",len(siblingsCount))

            #дістати інформацію про братів і сестер
            if (siblings.find_all("span")[1].get_text(strip=True) == "є"):
                siblingsArr = element.find(class_='passport__content-inner--brother').find_all("li")
                for sibling in siblingsArr:
                    childLink = sibling.find('a').get('href')
                    fullLink = "https://www.msp.gov.ua"+childLink
                    print(fullLink)
                    #відкрити посилання і дістати інформацію
                    responseSibl = requests.get(fullLink)
                    soupSibl = BeautifulSoup(responseSibl.text, "html.parser")
                    codeSibl = soupSibl.find_all(class_="container-fluid passport")
                    for childSib in codeSibl:
                        nameSibl = childSib.find(class_="passport__title-name")
                        if nameSibl:
                            print(nameSibl.get_text(strip=True))
                        else:
                            print("Ім'я не знайдено в цьому елементі.")  
                        

            
            
            #invalid
            invalid = element.find("ul").find_all("li")[3]
            if invalid:
                print("Invalid:",invalid.find_all("span")[1].get_text(strip=True))
            else:
                print("ivnalid not fount")
            
            #form
            form = element.find("ul").find_all("li")[4]
            if form:
                print("Form:",form.find_all("span")[1].get_text(strip=True))
            else:
                print("form not fount")
            
            #contacts
            contacts = element.find("ul").find_all("li")[5]
            if contacts:
                print("Contacts:",contacts.find_all("span")[1].get_text(strip=True))
            else:
                print("contacts not fount")

            print("##################################################")
            
            # Виводимо список братів і сестер
            #siblings = element.find(class_="passport__content-inner--brother").find_all("li")
            #for sibling in siblings:
            #    print(sibling)
            #print(f"Всього рідних братів: {len(siblings)}")

            # Перехід до наступного блоку
            next_step = element.find(class_="btn btn-default passport__footer-next")
            next_step = 0 #затичка
            if next_step:
                onclick_text = next_step.get("onclick")
                
                # Витягуємо відносний URL із onclick
                start_index = onclick_text.find("'") + 1
                end_index = onclick_text.rfind("'")
                relative_url = onclick_text[start_index:end_index]
                
                # Об'єднуємо з базовим URL
                base_url = "https://www.msp.gov.ua/children/"
                full_url = urljoin(base_url, relative_url)
                
                # Кодуємо для коректного URL
                encoded_url = quote(full_url, safe=":/?=&")
                parse_children_page(encoded_url)
                print("--------------------------------------")
                print("--------------------------------------")
            else:
                print("Кнопка переходу не знайдена")
    else:
        print("Не вдалося завантажити сторінку")

# Виклик функції з потрібним URL
#parse_children_page("https://www.msp.gov.ua/children/child.php?n=405181&form=*&male=*&age_from=3&age_to=3&region=*&brothers=*&needs=*")
parse_children_page("https://www.msp.gov.ua/children/child.php?n=405229&form=*&male=*&age_from=*&age_to=*&region=*&brothers=*&needs=*")


Юлія, 7 років
id дитини: 405229
photo src= ../files/children/05_children12.08.2024.zip/36630979.jpeg
Region: Донецька обл.
Siblings: є
siblings count 2
https://www.msp.gov.ua/children/child.php?n=405231
Ганна, 9 років
https://www.msp.gov.ua/children/child.php?n=405233
Андрій, 5 років
Invalid: немає
Form: національне усиновлення; опіка/піклування; прийомна сім'я та дитячий будинок сімейного типу;
Contacts: ССД Донецької ОДА: (06264) 8-52-23
##################################################
Кнопка переходу не знайдена


In [10]:
import pymysql
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="test"
)

# Виконуємо запит
try:
    with connection.cursor() as cursor:
        query = "SELECT * FROM persons LIMIT 5"
        cursor.execute(query)
        result = cursor.fetchall()
        for row in result:
            print(row)
finally:
    connection.close()

In [ ]:
"CREATE TABLE children (id INT PRIMARY KEY AUTO_INCREMENT,name VARCHAR(100) NOT NULL,age INT,status VARCHAR(50),disability BOOLEAN DEFAULT 0,has_siblings BOOLEAN DEFAULT 0,siblings_links TEXT);"


In [3]:
import pymysql
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="bastards"
)

# Виконуємо запит
try:
    with connection.cursor() as cursor:
        query = "CREATE TABLE adopt_children (\
        id INT PRIMARY KEY AUTO_INCREMENT,\
        childName VARCHAR(100) NOT NULL,\
        sex VARCHAR(20),\
        age INT,\
        photo_url VARCHAR(200) NOT NULL,\
        region TEXT,\
        siblings BOOLEAN,\
        siblings_ammount INT(2),\
        siblings_info TEXT,\
        invalid BOOLEAN,\
        form TEXT,\
        contacts TEXT\
        );"
        cursor.execute(query)
        result = cursor.fetchall()
        for row in result:
            print(row)
finally:
    connection.close()

"INSERT INTO children (childName, age, photo_url, region, siblings, siblings_ammount, siblings_info, invalid, form, contacts)VALUES    ('Олексій Іваненко', 10, 'https://example.com/photos/oleksiy.jpg', 'Київська область', 1, 2, 'Старший брат та молодша сестра', 0, 'Школа', 'Телефон: +380123456789'),    ('Марія Петренко', 8, 'https://example.com/photos/maria.jpg', 'Львівська область', 1, 1, 'Молодша сестра', 0, 'Домашнє навчання', 'Телефон: +380987654321'),    ('Іван Коваленко', 12, 'https://example.com/photos/ivan.jpg', 'Одеська область', 0, NULL, NULL, 0, 'Школа', 'Телефон: +380564738291'),    ('Анна Сидоренко', 9, 'https://example.com/photos/anna.jpg', 'Харківська область', 1, 3, 'Троє молодших братів', 0, 'Школа', 'Електронна пошта: anna@mail.com'),    ('Петро Василенко', 11, 'https://example.com/photos/petro.jpg', 'Вінницька область', 1, 2, 'Старша сестра та молодший брат', 1, 'Спеціальна школа', 'Телефон: +380456123789'),    ('Ольга Ткаченко', 7, 'https://example.com/photos/olga.j

In [16]:
import pymysql

# Підключення до бази даних
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="bastards"
)

# Виконуємо запит
try:
    with connection.cursor() as cursor:
        query = """
            INSERT INTO children (childName, age, photo_url, region, siblings, siblings_ammount, siblings_info, invalid, form, contacts) VALUES
            ('Олексій Іваненко', 10, 'https://example.com/photos/oleksiy.jpg', 'Київська область', 1, 2, 'Старший брат та молодша сестра', 0, 'Школа', 'Телефон: +380123456789'),
            ('Марія Петренко', 8, 'https://example.com/photos/maria.jpg', 'Львівська область', 1, 1, 'Молодша сестра', 0, 'Домашнє навчання', 'Телефон: +380987654321'),
            ('Іван Коваленко', 12, 'https://example.com/photos/ivan.jpg', 'Одеська область', 0, NULL, NULL, 0, 'Школа', 'Телефон: +380564738291'),
            ('Анна Сидоренко', 9, 'https://example.com/photos/anna.jpg', 'Харківська область', 1, 3, 'Троє молодших братів', 0, 'Школа', 'Електронна пошта: anna@mail.com'),
            ('Петро Василенко', 11, 'https://example.com/photos/petro.jpg', 'Вінницька область', 1, 2, 'Старша сестра та молодший брат', 1, 'Спеціальна школа', 'Телефон: +380456123789'),
            ('Ольга Ткаченко', 7, 'https://example.com/photos/olga.jpg', 'Житомирська область', 0, NULL, NULL, 0, 'Домашнє навчання', 'Електронна пошта: olga@mail.com'),
            ('Максим Гончар', 13, 'https://example.com/photos/maksym.jpg', 'Дніпропетровська область', 1, 1, 'Старший брат', 0, 'Школа', 'Телефон: +380321456987'),
            ('Аліна Руденко', 8, 'https://example.com/photos/alina.jpg', 'Полтавська область', 1, 2, 'Дві молодші сестри', 0, 'Школа', 'Телефон: +380789456123'),
            ('Богдан Яременко', 10, 'https://example.com/photos/bohdan.jpg', 'Івано-Франківська область', 0, NULL, NULL, 1, 'Спеціальна школа', 'Електронна пошта: bohdan@mail.com'),
            ('Катерина Шевченко', 9, 'https://example.com/photos/kateryna.jpg', 'Черкаська область', 1, 4, 'Дві старші сестри та два молодші брати', 0, 'Школа', 'Телефон: +380213456987')
        """
        cursor.execute(query)
        
    # Зберігаємо зміни
    connection.commit()
    print("Дані успішно додано!")
    
except Exception as e:
    print(f"Помилка: {e}")
finally:
    connection.close()


Дані успішно додано!


In [77]:
#############################################################
# РО
############################################################
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote
import re
import pymysql

def parse_children_page(url):
    response = requests.get(url)
    
    # Перевірка на успішність відповіді
    if response.status_code == 200:
        # Парсимо HTML за допомогою BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Знаходимо елементи, які містять інформацію (з актуальним класом)
        code = soup.find_all(class_="container-fluid passport")  # Актуалізуйте клас, якщо він інший

        # Перебір знайдених елементів у code
        for element in code:
            # Знаходимо ім'я
            nameAge = element.find(class_="passport__title-name")
                       
            if nameAge:
                nameAge = nameAge.get_text(strip=True)
            else:
                nameAge = ""
                
            # passport__title-number
            #id
            id = element.find(class_="passport__title-number").get_text(strip=True)
            number = re.search(r'\d+', id).group()

            if not number:
                number = 0
                
            # Виводимо фото
            photo = element.find(class_="js-lightBoxGallery").find('img').get('src')
            if not photo:
                photo = ""

            #region
            region = element.find("ul").find_all("li")[1]
            if region:
                region = region.find_all("span")[1].get_text(strip=True)
            else:
                region = ""
            
            #siblings
            siblings = element.find("ul").find_all("li")[2]
            
            #отримати кількість
            siblingsAmount = 0
            if (siblings.find_all("span")[1].get_text(strip=True) == "є"):
                siblingsCount = element.find(class_='passport__content-inner--brother').find_all("li")
                siblingsAmount = len(siblingsCount)
            haveSiblings = False
            
            #invalid
            invalid = element.find("ul").find_all("li")[3]
            
            if invalid:
                if invalid.find_all("span")[1].get_text(strip=True) == "є":
                    invalid = True
                else:
                    invalid = False
            
            #form
            form = element.find("ul").find_all("li")[4]
            if form:
                form = form.find_all("span")[1].get_text(strip=True)
            else:
                form = "форма не визначена"
            
            #contacts
            contacts = element.find("ul").find_all("li")[5]
            if contacts:
                contacts = contacts.find_all("span")[1].get_text(strip=True)
            else:
                contacts = "Контакти не визначено"

            print("______________________________________")

            name = nameAge.split(", ")[0]
            age = nameAge.split(" ")[1]
            if not age:
                age = "NONE"
            print('name - ', name)
            
            
           # print('age - ', age)
           # print('number - ', number)
           # print('photo - ', photo)
           # print('region - ', region)
           # print('siblings - ', haveSiblings)
           # print('howManySiblings - ', siblingsAmount)
           # print('invalid - ', invalid)
           # print('form - ', form)
           # print('contacts - ', contacts)

             #дістати інформацію про братів і сестер
            siblingsInfo = ""
            if (siblings.find_all("span")[1].get_text(strip=True) == "є"):
                haveSiblings = True
                siblingsArr = element.find(class_='passport__content-inner--brother').find_all("li")
                for sibling in siblingsArr:
                    childLink = sibling.find('a').get('href')
                    fullLink = "https://www.msp.gov.ua"+childLink
                    siblingsInfo += fullLink + "\n"
                    #відкрити посилання і дістати інформацію
                    responseSibl = requests.get(fullLink)
                    soupSibl = BeautifulSoup(responseSibl.text, "html.parser")
                    codeSibl = soupSibl.find_all(class_="container-fluid passport")
                    for childSib in codeSibl:
                        nameSibl = childSib.find(class_="passport__title-name")
                        if nameSibl:
                            siblingsInfo += nameSibl.get_text(strip=True)
                        
            print(siblingsInfo)              

            connection = pymysql.connect(
            host="localhost",
            user="root",
            password="root",
            database="bastards"
            )
            
            # Виконуємо запит
            
            try:
                with connection.cursor() as cursor:
                    query = """
                        INSERT INTO children (id, childName, age, photo_url, region, siblings, siblings_ammount, siblings_info, invalid, form, contacts) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        ON DUPLICATE KEY UPDATE contacts = VALUES(contacts)
                    """
                    values = (
                        number,
                        name, 
                        age, 
                        photo, 
                        region, 
                        haveSiblings, 
                        siblingsAmount, 
                        siblingsInfo, 
                        invalid, 
                        form, 
                        contacts
                    )
                    cursor.execute(query, values)
                    connection.commit()
            finally:
                connection.close()
            
            
            # Перехід до наступного блоку
            next_step = element.find(class_="btn btn-default passport__footer-next")
            #next_step = 0 #затичка
            if next_step:
                onclick_text = next_step.get("onclick")
                
                # Витягуємо відносний URL із onclick
                start_index = onclick_text.find("'") + 1
                end_index = onclick_text.rfind("'")
                relative_url = onclick_text[start_index:end_index]
                
                # Об'єднуємо з базовим URL
                base_url = "https://www.msp.gov.ua/children/"
                full_url = urljoin(base_url, relative_url)
                
                # Кодуємо для коректного URL
                encoded_url = quote(full_url, safe=":/?=&")
                parse_children_page(encoded_url)
                print("--------------------------------------")
                print("--------------------------------------")
            
    else:
        print("Не вдалося завантажити сторінку")

# Виклик функції з потрібним URL
#parse_children_page("https://www.msp.gov.ua/children/child.php?n=405181&form=*&male=*&age_from=3&age_to=3&region=*&brothers=*&needs=*")
parse_children_page("https://www.msp.gov.ua/children/child.php?n=431161&form=*&male=*&age_from=2&age_to=2&region=*&brothers=*&needs=*")


______________________________________
name -  Анастасія

______________________________________
name -  Олександр

______________________________________
name -  Катерина
https://www.msp.gov.ua/children/child.php?n=383913
Ольга, 4 роки
______________________________________
name -  Софія

______________________________________
name -  Денис

______________________________________
name -  Тимур
https://www.msp.gov.ua/children/child.php?n=435459
Станіслав, 11 роківhttps://www.msp.gov.ua/children/child.php?n=435467
Ілля, 7 роківhttps://www.msp.gov.ua/children/child.php?n=435474
Владислав, 4 роки
______________________________________
name -  Богдана

______________________________________
name -  Стефанія

______________________________________
name -  Артем
https://www.msp.gov.ua/children/child.php?n=441764
Олександр, 3 рокиhttps://www.msp.gov.ua/children/child.php?n=441770
Володимир, 5 роківhttps://www.msp.gov.ua/children/child.php?n=441771
Вікторія, 8 роківhttps://www.msp.gov.ua/child

KeyboardInterrupt: 

In [52]:
####################################
#для тестів
####################################
import pymysql

connection = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="bastards"
)

try:
    with connection.cursor() as cursor:
        query = """
            SELECT * FROM children 
            WHERE id = %s
        """
        values = (55559614,)  # Кортеж з одним значенням
        cursor.execute(query, values)
        
        # Отримуємо результати
        ifNumberExist = cursor.fetchall()
        
        if len(ifNumberExist) != 0:
            for row in ifNumberExist:    
                print("test")
        else:
            print("FALSE")

finally:
    connection.close()


FALSE
